In [59]:
import numpy as np
import pandas as pd


# ==============================================================================
# HÀM CÔNG CỤ
# ==============================================================================
def read_data_from_excel(file_path):
    try:
        df = pd.read_excel(file_path)
        if 'x' not in df.columns or 'y' not in df.columns: return None, None
        return df['x'].to_numpy(), df['y'].to_numpy()
    except Exception as e:
        print(f"Lỗi: {e}")
        return None, None


def nhandathuc(b_coeffs, c):
    """
    Nhân đa thức B(x) có hệ số b_coeffs với đơn thức (x - c).
    Hệ số được sắp xếp theo bậc tăng dần [a0, a1, a2, ...].
    """
    n = len(b_coeffs)
    a_coeffs = np.zeros(n + 1)
    if n > 0:
        a_coeffs[0] = -c * b_coeffs[0]
        for i in range(1, n):
            a_coeffs[i] = b_coeffs[i-1] - c * b_coeffs[i]
        a_coeffs[n] = b_coeffs[n-1]
    return a_coeffs

def get_standard_form_coeffs(newton_coeffs, x_nodes):
    """
    Chuyển đa thức Newton sang dạng chính tắc a0 + a1*x + ...
    """
    n = len(newton_coeffs)
    poly_coeffs = np.array([newton_coeffs[n-1]])
    for i in range(n - 2, -1, -1):
        poly_coeffs = nhandathuc(poly_coeffs, x_nodes[i])
        poly_coeffs[0] += newton_coeffs[i]
    return poly_coeffs

# ==============================================================================
# DỮ LIỆU ĐẦU VÀO
# ==============================================================================
x_data, y_data = read_data_from_excel("data/input_newton_moc_bat_ky.xlsx")
if x_data is None:
    exit()

n = len(x_data)
np.set_printoptions(suppress=True, precision=8)
pd.options.display.float_format = '{:.8f}'.format

# ==============================================================================
# BƯỚC 1: TÍNH VÀ HIỂN THỊ BẢNG TỶ SAI PHÂN
# ==============================================================================
print("=======================================================================")
print("BƯỚC 1: BẢNG TỶ SAI PHÂN (Định dạng giống Excel)")
print("=======================================================================")

dd_table = np.zeros((n, n))
dd_table[:, 0] = y_data
for j in range(1, n):
    for i in range(j, n):
        dd_table[i, j] = (dd_table[i, j-1] - dd_table[i-1, j-1]) / (x_data[i] - x_data[i-j])

df_display = pd.DataFrame(dd_table)
df_display.insert(0, 'X', x_data)
df_display.columns = ['X'] + ['Y'] + [f'TSP{i}' for i in range(1, n)]
df_display_str = df_display.round(8).astype(str).replace('0.00000000', '')
print(df_display_str.to_string(index=False))
print("-" * 71)

# ############################################################################
# PHẦN A: NỘI SUY NEWTON TIẾN
# ############################################################################
print("\n\n=======================================================================")
print("PHẦN A: NỘI SUY NEWTON TIẾN")
print("=======================================================================")

forward_coeffs = np.diag(dd_table)
print("\nA.1: Vector C_tiến (Hệ số Newton tiến):")
print(forward_coeffs)

print("\nA.2: Bảng Tích tiến")
forward_basis_table = np.zeros((n, n))
current_poly = np.array([1.0])
for i in range(n):
    if len(current_poly) <= n:
        padded_poly = np.pad(current_poly, (0, n - len(current_poly)))
        forward_basis_table[i, :] = np.flip(padded_poly)
    if i < n - 1:
        current_poly = nhandathuc(current_poly, x_data[i])
df_forward_basis = pd.DataFrame(forward_basis_table)
df_forward_basis.index = x_data
print(df_forward_basis.to_string(header=False))

# ############################################################################
# PHẦN B: NỘI SUY NEWTON LÙI
# ############################################################################
print("\n\n=======================================================================")
print("PHẦN B: NỘI SUY NEWTON LÙI")
print("=======================================================================")

backward_coeffs = dd_table[n-1, :]
print("\nB.1: Vector C_lùi (Hệ số Newton lùi):")
print(backward_coeffs)

print("\nB.2: Bảng Tích lùi")
backward_basis_table = np.zeros((n, n))
current_poly = np.array([1.0])
for i in range(n):
    if len(current_poly) <= n:
        padded_poly = np.pad(current_poly, (0, n - len(current_poly)))
        backward_basis_table[i, :] = np.flip(padded_poly)
    if i < n - 1:
        current_poly = nhandathuc(current_poly, x_data[n-1-i])
df_backward_basis = pd.DataFrame(backward_basis_table)
df_backward_basis.index = x_data[::-1]
print(df_backward_basis.to_string(header=False))

# ############################################################################
# PHẦN C: HỆ SỐ ĐA THỨC CHÍNH TẮC
# ############################################################################
print("\n\n=======================================================================")
print("PHẦN C: HỆ SỐ ĐA THỨC CHÍNH TẮC")
print("=======================================================================")

standard_coeffs = get_standard_form_coeffs(forward_coeffs, x_data)
# Numpy cần hệ số theo bậc giảm dần [a_n, a_{n-1}, ... a_0]
standard_coeffs_display = np.flip(standard_coeffs)

print("\nC.1: Các hệ số của đa thức nội suy dạng chính tắc (từ A₀ đến A_n):")
print(standard_coeffs)

# ############################################################################
# PHẦN D: TÍNH GIÁ TRỊ VÀ ĐẠO HÀM CẤP i
# ############################################################################
print("\n\n=======================================================================")
print("PHẦN D: TÍNH GIÁ TRỊ VÀ ĐẠO HÀM CẤP i")
print("=======================================================================")

# --- Nhập giá trị từ bàn phím ---
while True:
    try:
        x_val = float(input("Nhập giá trị x để tính toán: "))
        break
    except ValueError:
        print("Vui lòng nhập một số thực.")

while True:
    try:
        deriv_order = int(input("Nhập cấp đạo hàm i (nhập 0 để tính giá trị P(x)): "))
        if deriv_order >= 0:
            break
        else:
            print("Cấp đạo hàm phải là một số nguyên không âm.")
    except ValueError:
        print("Vui lòng nhập một số nguyên.")

# --- Tính toán ---
if deriv_order > 0:
    poly_derivative_coeffs = np.polyder(standard_coeffs_display, deriv_order)
else:
    poly_derivative_coeffs = standard_coeffs_display

result_value = np.polyval(poly_derivative_coeffs, x_val)

# --- In kết quả ---
print("\n--- KẾT QUẢ TÍNH TOÁN ---")

if deriv_order == 0:
    print(f"Giá trị của đa thức nội suy tại x = {x_val} là:")
    print(f"P({x_val}) = {result_value:.8f}")
elif deriv_order == 1:
    print(f"Giá trị của đạo hàm cấp 1 tại x = {x_val} là:")
    print(f"P'({x_val}) = {result_value:.8f}")
elif deriv_order == 2:
    print(f"Giá trị của đạo hàm cấp 2 tại x = {x_val} là:")
    print(f"P''({x_val}) = {result_value:.8f}")
else:
    print(f"Giá trị của đạo hàm cấp {deriv_order} tại x = {x_val} là:")
    print(f"P^({deriv_order})({x_val}) = {result_value:.8f}")

if deriv_order >= len(standard_coeffs_display):
    print(f"\nLưu ý: Đa thức nội suy có bậc {len(standard_coeffs_display)-1}. Đạo hàm cấp {deriv_order} sẽ bằng 0.")

BƯỚC 1: BẢNG TỶ SAI PHÂN (Định dạng giống Excel)
  X       Y         TSP1         TSP2        TSP3       TSP4        TSP5        TSP6       TSP7       TSP8
0.3  -0.739          0.0          0.0         0.0        0.0         0.0         0.0        0.0        0.0
0.6  -0.214         1.75          0.0         0.0        0.0         0.0         0.0        0.0        0.0
0.8    1.82        10.17        16.84         0.0        0.0         0.0         0.0        0.0        0.0
0.9   3.637        18.17  26.66666667 16.37777778        0.0         0.0         0.0        0.0        0.0
1.2  13.748  33.70333333  38.83333333 20.27777778 4.33333333         0.0         0.0        0.0        0.0
1.4  25.801       60.265  53.12333333 23.81666667 4.42361111  0.08207071         0.0        0.0        0.0
1.7  55.182  97.93666667  75.34333333      27.775 4.39814815 -0.02314815 -0.07515633        0.0        0.0
1.9  84.552       146.85  97.82666667 32.11904762 4.34404762  -0.0491823 -0.02002627 0.03445628

Nhập giá trị x để tính toán:  0
Nhập cấp đạo hàm i (nhập 0 để tính giá trị P(x)):  0



--- KẾT QUẢ TÍNH TOÁN ---
Giá trị của đa thức nội suy tại x = 0.0 là:
P(0.0) = -0.06572462
